## Debug the SparsDT
import lib

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [53]:
import torch
import numpy as np
import lib.models as model
import lib.util as util

In [4]:
Y = torch.rand([10,1600],dtype=torch.float32)

### Testing estimating alpha

In [5]:
SparsDT = model.ModelSparseDT(10,Y, num_samples=1e5)

================= estimating alpha =================


### Testing the optimizer

In [7]:
args = {
    'lr' : 0.1
}
SparsDT.set_optimizer('SGD', args)

### Testing  estim gamma

In [12]:
U = torch.randn([SparsDT.m_rows, 1000])
a = SparsDT._estimate_gamma(U)
a.shape

torch.Size([1000])

### Testing find U, norm alpha, Loss

In [18]:
U = SparsDT._findU(1000)
U.shape

torch.Size([10, 1000])

In [21]:
norm = SparsDT._norm_alpha(SparsDT.A, U)
norm.shape

torch.Size([1000])

In [25]:
gamma = SparsDT._estimate_gamma(U)
l = SparsDT.Loss(U,gamma)
l

tensor(0.1757, grad_fn=<MeanBackward1>)

### Testing fit

In [31]:
A = SparsDT.fit(max_iter=1e3, rand_U=True, num_col = 100)

### Testing a ground truth

In [51]:
m = 3 
n = 4
k = 1000
alpha = 1
beta = 0

In [54]:
A = util.Generate_A(m,n)
X = torch.tensor(util.Generate_alpha_random(alpha, beta, shape=(n, k)))